# 

# LCMS

## Setup

In [ ]:
library(readxl)
library(data.table)
library(ggplot2)
library(patchwork)
library(here)


here() starts at C:/Users/alosvendsen/Documents/R/manuscript

## Donor 3, AS0013B, Midazolam

Check peak status’ for any amounts below what below/above the limit of our standard curve.

In [ ]:
donor_3_data <- 
  readRDS(here("notebooks/LCMS/data_processed/AS0013.rds"))

# Check peak status'
unique(donor_3_data$peak_status)


[1] NA             "Response Low"

  ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
  sample_name        amount peak_status   metabolite           donor treatment   time_treatment     time_incubation   rep   mean_amount   control_mean   relative_amount   mean_relative_amount
  ------------- ----------- ------------- ------------------ ------- ----------- ---------------- ----------------- ----- ------------- -------------- ----------------- ----------------------
  AS0008_135      0.9118640 Response Low  Hydroxymidazolam         2 10 ng/ml    72                             0.5     3      1.530223       7.625022         0.1195884              0.2006845
                                                                     IL-6                                                                                                

  AS0008_197      0.8381536 Response Low  Hydroxymidazolam         2 10 ng/ml    120                            0.5     1      3.593515      13.685163         0.0612454              0.2625847
                                                                     IL-6                                                                                                
  ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
